In [1]:
import re
import requests
import lxml.html

In [3]:
def main():
    session = requests.Session()
    response = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

In [4]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url

In [6]:
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する。
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url':response.url, #URL
        'title':root.cssselect('#bookTitle')[0].text_content(), #タイトル
        'price':root.cssselect('.buy')[0].text.strip(), #価格(strip()で前後の空白を削除)
        'content':[normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')],
    }
    return ebook

In [7]:
def normalize_spaces(s):
    """
    連続する空白を1つのスペースに置き換え、前後の空白は削除した新しい文字列を取得する。
    """
    return re.sub(r'\s+',' ',s).strip()

In [8]:
if __name__=='__main__':
    main()

{'url': 'https://gihyo.jp/dp/ebook/2018/978-4-7741-9792-0', 'title': 'WEB+DB PRESS Vol.104', 'price': '1,598円', 'content': ['特集1 ［モダンなコードをギュッと凝縮！］イマドキPython入門 文法，機械学習，Web開発を一気に学ぼう', '特集2 いきなりiPhoneアプリ開発 カメラの写真を加工してTwitter投稿！', '特集3 はじめてのUnity シューティングゲームを作ろう！', '連載', 'コラム', 'Special Report', '特別広報']}
